### Problem Description

The k-armed bandit problem is a simple reinforcement learning problem where the environment contains only a single state. The agent is presented with k different actions that it can take, and each action gives out a reward generated from some stationary probability distribution. The agent starts out not knowing which action is the best action, so it must first explore all of the possibilities to figure out the expected reward of each action.

The real-life analogy of this problem is a gambler (bandit) that wants to maximize their money (reward) by playing the slot machines, but there are k slot machines, each with its own probabilistic payoff. The gambler must then figure out which slot machine is the most profitable.

In [1]:
# Setup

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import partial

def reset_seed(seed=42):
    np.random.seed(seed)

### The Environment

There will be two kinds of environments: discrete and continuous. A discrete bandit environment just means that there are some finite number of payoffs, each with its own probability (total probability must add up to one). A continuous bandit environment generates payoffs based on some probability distribution.

In [2]:
class DiscreteBandit():
    """
    A discrete bandit environment.
    
    Args:
        payoffs: (ndarray of shape [n_arms, n_payoffs]) the payoffs for each action
        probabilities (ndarray of shape [n_arms, n_payoffs]) the probability for every payoff of every action
    """
    def __init__(self, payoffs, probabilities):
        self.payoffs = payoffs
        self.probabilities = probabilities
        
    def step(self, x):
        return np.random.choice(self.payoffs[x], p=self.probabilities[x])

In [3]:
reset_seed(42)

payoffs = np.array([[-1, 0, 1],
                    [-2, 0, 5],
                    [0, 4, 8],
                    [-2, -1, 1]])

probabilities = np.array([[0.45, 0.1, 0.45],
                          [0.45, 0.1, 0.45],
                          [0.98, 0.01, 0.01],
                          [0.98, 0.01, 0.01]])

env = DiscreteBandit(payoffs, probabilities)

max_steps = 1000
total_reward = 0

for step in range(max_steps):
    total_reward += env.step(np.random.choice(len(payoffs)))  # choose a random action
    
print("Total reward:", total_reward)

Total reward: -111


In [4]:
class ContinuousBandit():
    """
    A continuous bandit environment.
    
    Args:        
    """
    def __init__(self, reward_distributions):
        self.reward_distributions = reward_distributions
        
    def step(self, x):
        return self.reward_distributions[x]()

In [5]:
reset_seed(42)

mu = [np.random.uniform(-10, 10) for _ in range(5)]
sigma = [np.random.uniform(0, 5) for _ in range(5)]
reward_distributions = [partial(np.random.normal, m, s) for m, s in zip(mu, sigma)]

for m, s in zip(mu, sigma):
    print("Mean: %.2f \t Sigma: %.2f" % (m, s))

env = ContinuousBandit(reward_distributions)

max_steps = 1000
total_reward = 0

for step in range(max_steps):
    total_reward += env.step(np.random.choice(len(reward_distributions)))  # choose a random action
    
print("Total reward: %.2f" % total_reward)

Mean: -2.51 	 Sigma: 0.78
Mean: 9.01 	 Sigma: 0.29
Mean: 4.64 	 Sigma: 4.33
Mean: 1.97 	 Sigma: 3.01
Mean: -6.88 	 Sigma: 3.54
Total reward: 848.69


### The Greedy Policy

Because we are interested in creating an artificial intelligence, our agent needs to do better than just taking a random action at each step. The agent needs to be able to make decisions based on something. In RL lingo, we call this decision-making ability a "policy". The simplest non-random policy is probably the greedy policy, where the agent chooses what it thinks as the optimal action at each given step.

In [6]:
reset_seed(42)

n = 5

mu = [np.random.uniform(-10, 10) for _ in range(n)]
sigma = [np.random.uniform(0, 5) for _ in range(n)]
reward_distributions = [partial(np.random.normal, m, s) for m, s in zip(mu, sigma)]

# Store rewards obtained at each step. This acts as a "memory" and can be used to make decisions.
rewards = [[] for _ in range(n)]
total_rewards = [0 for _ in range(n)]
n_tries = [0 for _ in range(n)]

def expected_reward(a):
    if n_tries[a] == 0:
        return 0
    else:
        return total_rewards[a] / n_tries[a]
    
def greedy_policy():
    return np.argmax([expected_reward(a) for a in range(n)])

env = ContinuousBandit(reward_distributions)

max_steps = 1000

for step in range(max_steps):
    a = greedy_policy()
    reward = env.step(a)
    rewards[a].append(reward)
    total_rewards[a] += reward
    n_tries[a] += 1
    
print("Total reward: %.2f" % sum(total_rewards))

Total reward: 9008.13


### T

The greedy policy has an obvious weakness, namely it is short-sighted. A typical reinforcement learning environment is uncertain, the agent almost always starts out not knowing which action is the best and what is the expected reward of each action. It faces the dilemma of choosing between what it thinks is the optimal action (exploitation), and choosing some other action in the hope of improving its knowledge so that it can make better decisions in the future (exploration). This is called the exploration-exploitation tradeoff.

In the greedy example above, the agent performs no exploration at all, it always chooses what it thinks as the best action at each step. As a result, it stops looking when it finds an action that yields a positive reward (on average), and doesn't consider the possibility that there might be some other action that gives better rewards. One way to integrate exploration into the greedy policy is to perform each action some number of times to gather data before going greedy all the way until the end. Since there are 1000 time steps, one option is to perform each action 20 times for the first 100 time steps, then choose the best action for the other 900. But this approach seems arbitrary, and it doesn't guarantee convergence (there's a non-zero chance that 20 data points per action isn't enough to determine the true expected reward).

One exploration policy that has been around since the dawn of RL is the epsilon-greedy policy. It works by setting some exploration coefficient, epsilon, that determines the probability of choosing a random action instead of taking the greedy action. So, if epsilon is set to 0.1, then at any given time step the agent has 10% probability of choosing a random action and 90% probability of choosing the greedy action. This policy guarantees convergence, each action will be taken an infinite number of times as time goes to infinity.

In [7]:
reset_seed(42)

n = 5

mu = [np.random.uniform(-10, 10) for _ in range(n)]
sigma = [np.random.uniform(0, 5) for _ in range(n)]
reward_distributions = [partial(np.random.normal, m, s) for m, s in zip(mu, sigma)]

# Store rewards obtained at each step. This acts as a "memory" and can be used to make decisions.
rewards = [[] for _ in range(n)]
total_rewards = [0 for _ in range(n)]
n_tries = [0 for _ in range(n)]

def expected_reward(a):
    if n_tries[a] == 0:
        return 0
    else:
        return total_rewards[a] / n_tries[a]
    
def epsilon_greedy(epsilon):
    if np.random.rand() < epsilon:
        return np.random.randint(n)  # random action
    else:
        return np.argmax([expected_reward(a) for a in range(n)])  # greedy action

env = ContinuousBandit(reward_distributions)

max_steps = 1000
epsilon = 0.1

for step in range(max_steps):
    a = epsilon_greedy(epsilon)
    reward = env.step(a)
    rewards[a].append(reward)
    total_rewards[a] += reward
    n_tries[a] += 1
    
print("Total reward: %.2f" % sum(total_rewards))

Total reward: 8036.72
